In [1]:
!unzip ./data/titanic.zip -d ./data/

Archive:  ./data/titanic.zip
  inflating: ./data/gender_submission.csv  
  inflating: ./data/test.csv         
  inflating: ./data/train.csv        


In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

from tensorflow.keras.utils import to_categorical

In [9]:
train_pd = pd.read_csv('./data/train.csv')
train_pd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train_pd = train_pd[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
train_pd.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
train_pd.dropna(inplace=True)
train_pd.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [12]:
train_pd.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,0.404494,2.240169,29.642093,0.514045,0.432584,34.567251
std,0.491139,0.836854,14.492933,0.930692,0.854181,52.938648
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,20.000000,0.000000,0.000000,8.050000
50%,0.000000,2.000000,28.000000,0.000000,0.000000,15.645850
75%,1.000000,3.000000,38.000000,1.000000,1.000000,33.000000
max,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200


In [16]:
pclass = train_pd['Pclass'].to_numpy()
as_categorical_pclass = to_categorical(pclass - 1, num_classes=3)
as_categorical_pclass

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [17]:
def sex_str_to_binary(s):
    if s == "male":
        return 1
    else:
        return 0


sex = train_pd['Sex']

as_binary_sex = sex.apply(sex_str_to_binary).to_numpy()
as_binary_sex[:10]

array([1, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [18]:
age_np = train_pd['Age'].to_numpy()
age_np = (age_np - train_pd['Age'].mean()) / train_pd['Age'].std()
age_np[:10]

array([-0.52729787,  0.57668847, -0.25130129,  0.36969103,  0.36969103,
        1.68067481, -1.9072808 , -0.18230214, -1.07929105, -1.76928251])

In [19]:
sibsp = train_pd['SibSp']
sibsp_np = sibsp.to_numpy()
sibsp_np = (sibsp_np - train_pd['SibSp'].mean()) / train_pd['SibSp'].std()
sibsp_np[:10]

array([ 0.52214373,  0.52214373, -0.55232545,  0.52214373, -0.55232545,
       -0.55232545,  2.67108207, -0.55232545,  0.52214373,  0.52214373])

In [20]:
parch = train_pd['Parch']
parch = parch.to_numpy()
parch = (parch - train_pd['Parch'].mean()) / train_pd['Parch'].std()
parch[:10]

array([-0.50643136, -0.50643136, -0.50643136, -0.50643136, -0.50643136,
       -0.50643136,  0.66428009,  1.83499154, -0.50643136,  0.66428009])

In [21]:
fare = train_pd['Fare'].to_numpy()
fare = (fare - train_pd['Fare'].mean()) / train_pd['Fare'].std()
fare[:10]

array([-0.51601717,  0.69355848, -0.50326656,  0.35007975, -0.50090534,
        0.32670363, -0.25486581, -0.44266244, -0.08493703, -0.33750864])

In [23]:
train_Y = train_pd['Survived'].to_numpy()
train_Y[:10]

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 1])

In [24]:
train_X = np.concatenate((as_categorical_pclass,
                          as_binary_sex.reshape(-1, 1),
                          age_np.reshape(-1, 1),
                          sibsp_np.reshape(-1, 1),
                          parch.reshape(-1, 1),
                          fare.reshape(-1, 1)), axis=1)
train_X

array([[ 0.        ,  0.        ,  1.        , ...,  0.52214373,
        -0.50643136, -0.51601717],
       [ 1.        ,  0.        ,  0.        , ...,  0.52214373,
        -0.50643136,  0.69355848],
       [ 0.        ,  0.        ,  1.        , ..., -0.55232545,
        -0.50643136, -0.50326656],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.55232545,
        -0.50643136, -0.08627442],
       [ 1.        ,  0.        ,  0.        , ..., -0.55232545,
        -0.50643136, -0.08627442],
       [ 0.        ,  0.        ,  1.        , ..., -0.55232545,
        -0.50643136, -0.50657227]])

In [25]:
model = LogisticRegression(random_state=0).fit(train_X, train_Y)
model.score(train_X, train_Y)

0.800561797752809

In [26]:
test_pd = pd.read_csv('./data/test.csv')
test_pd

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [27]:
test_pd = test_pd[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
test_pd

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,male,34.5,0,0,7.8292
1,893,3,female,47.0,1,0,7.0000
2,894,2,male,62.0,0,0,9.6875
3,895,3,male,27.0,0,0,8.6625
4,896,3,female,22.0,1,1,12.2875
...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500
414,1306,1,female,39.0,0,0,108.9000
415,1307,3,male,38.5,0,0,7.2500
416,1308,3,male,NaN,0,0,8.0500


In [28]:
test_pd = test_pd.fillna(test_pd['Age'].mean())
len(test_pd)

418

In [29]:
pclass = test_pd['Pclass'].to_numpy()
as_categorical_pclass = to_categorical(pclass - 1, num_classes=3)
as_categorical_pclass[:10]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [30]:
sex = test_pd['Sex']
as_binary_sex = sex.apply(sex_str_to_binary).to_numpy()
as_binary_sex[:10]

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1])

In [31]:
age_np = test_pd['Age'].to_numpy()
age_np = (age_np - train_pd['Age'].mean()) / train_pd['Age'].std()
age_np[:10]

array([ 0.33519146,  1.19768079,  2.23266799, -0.18230214, -0.52729787,
       -1.07929105,  0.0246953 , -0.25130129, -0.80329446, -0.59629702])

In [32]:
sibsp = test_pd['SibSp']
sibsp_np = sibsp.to_numpy()
sibsp_np = (sibsp_np - train_pd['SibSp'].mean()) / train_pd['SibSp'].std()
sibsp_np[:10]

array([-0.55232545,  0.52214373, -0.55232545, -0.55232545,  0.52214373,
       -0.55232545, -0.55232545,  0.52214373, -0.55232545,  1.5966129 ])

In [33]:
parch = test_pd['Parch']
parch = parch.to_numpy()
parch = (parch - train_pd['Parch'].mean()) / train_pd['Parch'].std()
parch[:10]

array([-0.50643136, -0.50643136, -0.50643136, -0.50643136,  0.66428009,
       -0.50643136, -0.50643136,  0.66428009, -0.50643136, -0.50643136])

In [34]:
fare = test_pd['Fare'].to_numpy()
fare = (fare - train_pd['Fare'].mean()) / train_pd['Fare'].std()
fare[:10]

array([-0.5050762 , -0.52073962, -0.4699733 , -0.48933534, -0.42085985,
       -0.47870983, -0.50885416, -0.10516422, -0.51641008, -0.1967797 ])

In [35]:
test_X = np.concatenate((as_categorical_pclass,
                         as_binary_sex.reshape(-1, 1),
                         age_np.reshape(-1, 1),
                         sibsp_np.reshape(-1, 1),
                         parch.reshape(-1, 1),
                         fare.reshape(-1, 1)), axis=1)
test_X

array([[ 0.        ,  0.        ,  1.        , ..., -0.55232545,
        -0.50643136, -0.5050762 ],
       [ 0.        ,  0.        ,  1.        , ...,  0.52214373,
        -0.50643136, -0.52073962],
       [ 0.        ,  1.        ,  0.        , ..., -0.55232545,
        -0.50643136, -0.4699733 ],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.55232545,
        -0.50643136, -0.51601717],
       [ 0.        ,  0.        ,  1.        , ..., -0.55232545,
        -0.50643136, -0.50090534],
       [ 0.        ,  0.        ,  1.        , ...,  0.52214373,
         0.66428009, -0.23062454]])

In [36]:
predictions = model.predict(test_X)
passenger_ids = test_pd['PassengerId'].to_numpy()
sub_df = pd.DataFrame({'PassengerId': passenger_ids, 'Survived':predictions})

In [37]:
sub_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [38]:
sub_df.to_csv('./out/sub.csv')

In [39]:
submission_df = pd.read_csv('./data/gender_submission.csv')
submission_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [42]:
mean_squared_error(submission_df['Survived'], sub_df['Survived'])

0.10047846889952153